In [1]:
import skbeam.core.utils as utils

import numpy as np

In [2]:
num_levels = 2
num_bufs = 8
# Convert from num_levels, num_bufs to lag frames.
tot_channels, lag_steps, dict_lag = utils.multi_tau_lags(num_levels, num_bufs)

In [9]:
dict_lag

{1: array([0, 1, 2, 3, 4, 5, 6, 7]), 2: [8, 10, 12, 14]}

In [10]:
norm = {key: [0] * len(dict_lag[key]) for key in (dict_lag.keys())}
lev_len = np.array([len(dict_lag[i]) for i in (dict_lag.keys())])

In [11]:
type(norm)

dict

In [12]:
type(lev_len)


numpy.ndarray

In [13]:
num_lev = 1
num_buf = 10  # must be even
x_dim = 10
y_dim = 10
stack = 10
imgs = np.random.randint(1, 3, (stack, x_dim, y_dim))
rois = np.zeros_like(imgs[0])
# make sure that the ROIs can be any integers greater than 1.
# They do not have to start at 1 and be continuous
rois[0:x_dim//10, 0:y_dim//10] = 5
rois[x_dim//10:x_dim//5, y_dim//10:y_dim//5] = 3

In [14]:
from skbeam.core.correlation import two_time_corr

In [15]:
two_time = two_time_corr(rois, imgs,  10,
                             8, num_lev)


In [16]:
import skbeam.core.roi as roi
label_array, pixel_list = roi.extract_label_indices(rois)

In [18]:
 num_levels, num_bufs = 3, 4
tot_channels, lags, dict_lags = utils.multi_tau_lags(num_levels, num_bufs)
beta = 0.5
relaxation_rate = 10.0
baseline = 1.0

In [65]:
import skbeam.core.correlation as corr
num_lev = 1
num_buf = 10  # must be even
x_dim = 10
y_dim = 10
stack = 10
imgs = np.random.randint(1, 10, (stack, x_dim, y_dim))
rois = np.zeros_like(imgs[0])
# make sure that the ROIs can be any integers greater than 1.
# They do not have to start at 1 and be continuous
rois[0:x_dim//10, 0:y_dim//10] = 5
rois[x_dim//10:x_dim//5, y_dim//10:y_dim//5] = 3

two_time = two_time_corr(rois, imgs, stack,
                             num_buf, num_lev)

one_time = corr.one_time_from_two_time(two_time[0])

In [66]:
import matplotlib.pyplot as plt
plt.plot(one_time[1,:])
plt.show()

In [67]:
n_list= [2, 3, 4,5]

In [68]:
n_list[0], n_list[2], n_list[1], n_list[3]

(2, 4, 3, 5)

In [69]:
g2 , lag= corr.multi_tau_auto_corr??


In [ ]:
g2 , lag= corr.multi_tau_auto_corr

In [71]:
g2 , lag= corr.multi_tau_auto_corr(num_lev, num_buf, rois, imgs)

In [72]:
g2.shape

(10, 2)

In [73]:
y= g2.T

In [74]:
y.shape

(2, 10)

In [75]:
g2, y

(array([[ 1.15699537,  1.25514403],
        [ 0.97276596,  1.05612245],
        [ 1.02768549,  1.06511176],
        [ 0.89196676,  0.99461279],
        [ 1.02162162,  1.02769231],
        [ 0.91235632,  1.02796053],
        [ 1.13043478,  0.92873563],
        [ 1.02666667,  0.93650794],
        [ 1.03636364,  0.96666667],
        [ 1.        ,  1.        ]]),
 array([[ 1.15699537,  0.97276596,  1.02768549,  0.89196676,  1.02162162,
          0.91235632,  1.13043478,  1.02666667,  1.03636364,  1.        ],
        [ 1.25514403,  1.05612245,  1.06511176,  0.99461279,  1.02769231,
          1.02796053,  0.92873563,  0.93650794,  0.96666667,  1.        ]]))

In [76]:
two_time[0].shape

(2, 10, 10)

In [77]:
y.shape

(2, 10)

In [78]:
def get_four_time_from_two_time(g12, g2, time_ran=None):
    """
    Get four-time correlation function from two correlation function
    namely, calculate the deviation of each diagonal line of g12 to get
    four-time correlation function.

    This function computes four-time correlation
    Original code : author: Yugang Zhang

    Parameters
    ----------
    g12: array
        two time correlation results
        shape is (num ROI's, num images, num images)
    g2 : array
        one time correlation results
        shape is either (num images, num ROI's) or (num of lag steps, num ROI's)
        (see the notes in lazy_one_time)
    time_ran : list
        time range, give the desired time ranges for t1 and t2 the times used for
        two time correlation, optional
        len(list) = 4
        e.g., [x1, x2, y1, y2]

    Return
    ------
    g4 : array
         four-time correlation
         shape is (num ROI's, num images)

    Notes
    -----
    The four-time correlation function is defined as

    :math ::
        C(q, t_1, t_2) = \frac{<I(q, t_1)I(q, t_2)>_pix }{<I(q, t_1)>_pix <I(q, t_2)>_pix}

    Here, the ensemble averages are performed over many pixels of detector,
    all having the same q value. The average time or age is equal to (t1+t2)/2,
    measured by the distance along the t1 = t2 diagonal.
    The time difference t = |t1 - t2|, with is distance from the t1 = t2
    diagonal in the perpendicular direction.
    In the equilibrium system, the two-time correlation functions depend only
    on the time difference t, and hence the two-time correlation contour lines
    are parallel.

     References:  text [1]_, text [2]_

    .. [1] A. Duri,H. Bissig, V. Trappe and L. Cipelletti,
        "Time-resolved-correlation measurements of temporally heterogeneous
         dynamics," Phys. Rev. E., vol 72, p 05141(1-17), 2005.

    .. [2] P. A. Lemieux and D. J. Durian, "Investigating non-Gaussian
       scattering process by using nth-order intensity correlation functions",
       J. Opt. Soc. Am. A., vol 16, p 972-978, 2006.

    """
    g4 = []
    t = 0
    for x, y in zip(g2, g12):
        t += 1
        print t
        temp = []
        if time_ran is not None:
            y = y[time_ran[0]:time_ran[1], time_ran[3]:time_ran[4]]
        norm = (x[0] - 1)**2
        for tau in range(y.shape[1]):
            d_ = np.diag(y, k=int(tau))
            print d_
            d = d_[np.where(d_ != 1)]
            print "d", d
            g41 = (d.std()) ** 2 / norm
            temp.append(g41)
        temp = np.array(temp).reshape(len(temp), 1)
        if t == 1:
            g4 = temp
        else:
            g4 = np.hstack([g4, temp])

    return g4


In [79]:
for x,y  in zip(g2, two_time[0]):
        print (x[0]-1)**2

0.0246475468352
0.000741693073789


In [80]:
g2 = g2.T

In [81]:
g2

array([[ 1.15699537,  0.97276596,  1.02768549,  0.89196676,  1.02162162,
         0.91235632,  1.13043478,  1.02666667,  1.03636364,  1.        ],
       [ 1.25514403,  1.05612245,  1.06511176,  0.99461279,  1.02769231,
         1.02796053,  0.92873563,  0.93650794,  0.96666667,  1.        ]])

In [83]:
two_time[0]

array([[[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,

In [82]:
g4 = get_four_time_from_two_time(two_time[0], g2, time_ran=None)

1
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.]
d []
[ 1.  1.  1.]
d []
[ 1.  1.]
d []
[ 1.]
d []
2
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.  1.]
d []
[ 1.  1.  1.  1.]
d []
[ 1.  1.  1.]
d []
[ 1.  1.]
d []
[ 1.]
d []


In [64]:
g4

array([[ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan],
       [ nan,  nan]])

In [1]:
def get_four_time_from_two_time(g12, g2, time_ran=None):
    """
    Get four-time correlation function from two correlation function
    namely, calculate the deviation of each diagonal line of g12 to get
    four-time correlation function.

    This function computes four-time correlation
    Original code : author: Yugang Zhang

    Parameters
    ----------
    g12: array
        two time correlation results
        shape is (num ROI's, num images, num images)
    g2 : array
        one time correlation results
        shape is either (num images, num ROI's) or (num of lag steps,
        num ROI's) (see the notes in lazy_one_time)
    time_ran : list
        time range, give the desired time ranges for t1 and t2 the times
        used for two time correlation, optional
        len(list) = 2
        e.g., [x1, x2]

    Return
    ------
    g4 : array
         four-time correlation
         shape is (num ROI's, num images)

    Notes
    -----
    The four-time correlation function is defined as

    :math ::
        g4(q,T,t) = \frac{<I(q, 0)I(q, T)I(q, t)I(q, t+ T)>}{<I(q, 0)I(q,T)>^2}

        where T and t are the delays

    Here, the ensemble averages are performed over many pixels of detector,
    all having the same q value.

     References:  text [1]_, text [2]_

    .. [1] A. Duri,H. Bissig, V. Trappe and L. Cipelletti,
        "Time-resolved-correlation measurements of temporally heterogeneous
        dynamics," Phys. Rev. E., vol 72, p 05141(1-17), 2005.

    .. [2] P. A. Lemieux and D. J. Durian, "Investigating non-Gaussian
       scattering process by using nth-order intensity correlation functions",
       J. Opt. Soc. Am. A., vol 16, p 972-978, 2006.
    """
    g4 = []
    for x, y in zip(g2, g12):
        temp = []
        if rois is not None:
            y = y[time_ran[0]:time_ran[1], time_ran[0]:time_ran[1]]
        norm = (x[0] - 1)**2
        for tau in range(y.shape[1]):
            d_ = np.diag(y, k=int(tau))
            d = d_[np.where(d_ != 1)]
            g41.append((d.std()) ** 2 / norm)
        g41 = np.array(temp).reshape(len(temp), 1)
        if q == 0:
            g4 = temp
        else:
            g4 = np.hstack([g4, temp])

    return g4
